BOT Using Websockets

In [ ]:
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
from cvzone import overlayPNG
import cv2
import websocket

ws = websocket.WebSocket()
ws.connect("ws://192.168.0.106:81")


fpsReader = FPS(avgCount=30)


# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)

# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.6, minTrackCon=0.6)



# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()
    H,W,_ = img.shape

    roix1, roiy1, roix2, roiy2 = 0, 0, W, int(H*0.80)
    imgroi = img[roiy1:roiy2, roix1: roix2]
    cv2.rectangle(img, (roix1, roiy1), (roix2, roiy2), (0,200,0), 2)

    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, imgroi = detector.findHands(imgroi, draw=True, flipType=True)

    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")

        # Count the number of fingers up for the first hand
        fingers1 = detector.fingersUp(hand1)

        # Calculate distance between specific landmarks on the first hand and draw it on the image
        #length, info, img = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], img, color=(255, 0, 255), scale=10)
        
        if handType1 == "Right": 
            if fingers1.count(1) == 5:
                data_to_send = 'F'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "FORWARD"
        
            elif fingers1.count(1) == 2:
                data_to_send = 'L'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "LEFT"

            elif fingers1.count(1) == 3:
                data_to_send = 'R'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "RIGHT" 
            elif fingers1.count(1) == 4:
                data_to_send = 'B'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "REVERSE" 

            else: 
                data_to_send = 'S'  
                ws.send(data_to_send)
                #ws.close() 
                stateText = "STOP" 
            
            

        if handType1 == "Left": 
            if fingers1.count(1) == 5:
                #arduino.sendData([0])
                data_to_send = 'B'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "REVERSE"

            elif fingers1.count(1) == 2:
            #if fingers1[1]==1 and fingers1[2] == 1 :
                data_to_send = 'L'  
                ws.send(data_to_send)
                #ws.close()
                stateText = "LEFT" 


            else: 
                data_to_send = 'S'  
                ws.send(data_to_send)
                #ws.close() 
                stateText = "STOP" 

        putTextRect(img, text=stateText, pos=(int(W*0.015), 200), scale=1.3, thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
        


            
                

        # Check if a second hand is detected
        if len(hands) == 2:
            # Information for the second hand
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Count the number of fingers up for the second hand
            fingers2 = detector.fingersUp(hand2)
         
    else: 
         data_to_send = 'S'  
         ws.send(data_to_send)   


    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window
    cv2.imshow("Image", img)

    # Wait for 1 millisecond to check for any user input, keeping the window open
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()